In [1]:
%reset -f
import os
print(os.getcwd())
os.chdir(f"{os.getcwd()}/insectrec/")
print(os.getcwd())

/home/kalfasyan/projects/yannis_impy
/home/kalfasyan/projects/yannis_impy/insectrec


In [ ]:
import numpy as np
seed = 42
np.random.seed(seed)
import pandas as pd
import os, shutil, glob, cv2, sys, argparse, git
from natsort import natsorted
from utils import clean_folder, get_plate_names, export_labels_2019, SAVE_DIR, read_plate
from tqdm import tqdm
repo = git.Repo('.', search_parent_directories=True)
created_data_path = f'{repo.working_tree_dir}/insectrec/created_data'


In [3]:
created_data_path

'/home/kalfasyan/projects/yannis_impy/insectrec/created_data'

In [4]:
# parser = argparse.ArgumentParser()
# parser.add_argument("--datadir", help="directory of sticky plate images")
# args = parser.parse_args()
# assert isinstance(args.datadir, str) and os.path.isdir(args.datadir), 'Provide a valid path'
# if not len(args.datadir): # /home/kalfasyan/data/images/sticky_plates/
# 	raise ValueError("Please provide a datadir argument.")
# datadir = '/home/kalfasyan/data/images/sticky_plates/2019/'
datadir = '/home/kalfasyan/data/images/sticky_plates/2020/'

In [5]:
# CREATING NECESSARY DIRECTORIES FOR THE PROJECT
path_annotations = f'{created_data_path}/annotations/'
path_images = f'{created_data_path}/images/'
path_voc_annotations = f'{created_data_path}/voc_annotations/'
path_impy_crops_export = f'{created_data_path}/impy_crops_export/'
path_images_augmented = f'{created_data_path}/images_augmented/'
path_weights = f'{created_data_path}/weights/'
path_logs = f'{created_data_path}/logs/'
for path in [path_annotations, path_images, path_voc_annotations, 
			path_impy_crops_export, path_weights, path_logs, path_images_augmented]:
	if not os.path.isdir(path):
		os.mkdir(path)	

In [6]:
include_herent = True
yolo_to_voc = True # In the end of the script, yolo annotations get converted to voc
extract_boxes = True # Only works if above is true. Bounding boxes extracted and saved as images
clean = True # Deleting previous data created here (i.e. except of logs and weights)
if clean:
	print(f'Cleaning directories..')
	clean_folder(path_annotations)
	clean_folder(path_images)
	clean_folder(path_voc_annotations)
	os.system(f'rm -rf {path_impy_crops_export}*')
	os.system(f'rm -rf {path_images_augmented}*')
	os.system(f'rm {created_data_path}/df_*')
	os.system(f'rm {created_data_path}/class_mapping.csv')

Cleaning directories..


In [7]:
# Get name data from the sticky plates (their names)
BASE_DATA_DIR = f"{datadir}"
year = datadir.split('/')[-2] if datadir.endswith('/') else datadir.split('/')[-2]
print(year)
assert year in ['2019','2020'], 'Wrong year given'
plates = get_plate_names(year, base_dir=BASE_DATA_DIR)

2020


In [8]:
# Create classes.txt for yolo annotations 
# and a class_mapping.csv with the human readable labels
export_labels_2019(base_dir=SAVE_DIR)
class_map = pd.read_csv('created_data/class_mapping.csv')
assert len(class_map), "Couldn't read class mapping"
sub = class_map[['class', 'class_encoded']].drop_duplicates()
nan_code = sub[sub['class'].isnull()]['class_encoded'].iloc[0]

Found these excel annotation files: ['./created_data/results_20200212_W40 .xlsx', './created_data/results_20200810_w31&Racour3aug .xlsx', './created_data/results_20200724_w29&Racour2en13juli .xlsx', './created_data/results_datageleplaten_w00.xlsx', './created_data/results_20190730_20190806.xlsx', './created_data/results_2019_annotations_yannis.xlsx', './created_data/results_20190821.xlsx', './created_data/results_20191004_W38_en_W39.xlsx', './created_data/results_20191001_W35.xlsx', './created_data/results_20200129_W41 .xlsx', './created_data/results_20190809_ W31.xlsx', './created_data/results_20191001_W36.xlsx', './created_data/results_may_june.xlsx', './created_data/results_20191001_W34.xlsx', './created_data/results _20200707_w26 & Herent Controle, val 1, 2, 3 en delta.xlsx', './created_data/results_20191001_W37.xlsx', './created_data/results_20180806_W29.xlsx']
Processing annotation file: ./created_data/results_20200212_W40 .xlsx
Processing annotation file: ./created_data/results_

In [9]:

# Create a dataframe to save some statistics about the plates
# such as the number of nans and number of unique insects per plate
short_platenames = pd.Series(plates).apply(lambda x: x.split("/")[-1][:-4])
df_stats = pd.DataFrame(columns=['nr_nans','unique_insects','annotated'], index=short_platenames)
all_specs = []

annotated_plates, incomplete_plates = [], []
# Extra pixels around the image to crop
extra_pixels = 20

# Plates to ignore, since they were found to contain bad data (blurred/misclassified etc.)
bad_plates = []
# "['brainlal_w27_A_58_160_1-15 s_11_48 mm_Manual_Manual_6240 x 4160', 
# 			'brainelal_8719_B_81_160_1-15 s_11_48 mm_Manual_Manual_6240 x 4160',
# 			'kampen_w25_C_72_160_1-15 s_11_48 mm_Manual_Manual_6240 x 4160',
# 			'kampen_w25_B_71_160_1-15 s_11_48 mm_Manual_Manual_6240 x 4160']"

# Loop through all plates and nested loop through all insects in the plates
for p, platename in tqdm(enumerate(plates)):
    # Skip very early plates from herent that were imaged with old Canon 
    if not include_herent:
        if platename.split('/')[-1].startswith('her'):
            print("SKIPPING HERENT PLATE")
            continue
    if platename.split('/')[-1][:-4] in bad_plates:
        print("SKIPPING BAD PLATE")
        continue

    pname = platename.split('/')[-1][:-4] # defining the platename
    if 'empty' in pname:
        continue
    plate_img = read_plate(platename) # reading the plate image

    H,W,_ = plate_img.shape

    spec = pd.read_csv(plates[p][:-4] + '.txt', sep="\t") # reading the specifications of the plate

    if p == 0: # fetching column names (only once needed)
        columns = [ii for ii in spec.columns if ii.endswith('.1')]
        colextensions = ['index', 'name plate', 'R','G','B']
        columns.extend(colextensions) # adding extra columns

    spec = spec[columns]
    spec.dropna(axis=0, how='any', inplace=True) # cleaning up

    # REPLACING COMMAS WITH DOTS
    tmp = [spec[col].str.replace(',','.').astype(float) for col in columns if col not in colextensions]
    spec = pd.concat(tmp, axis=1) # changing type to float
    del tmp

    # ADDING YOLO AND HUMAN-READABLE ANNOTATION TO COLUMNS
    cmap = class_map[class_map['platename'] == pname].drop_duplicates(subset='idx', keep='first')
    sub_map = cmap[['idx','class_encoded']].set_index('idx')
    sub_map2 = cmap[['idx','class']].set_index('idx')
    spec['yolo_class'] = sub_map
    spec['normal_class'] = sub_map2

    # REMOVING UNWANTED CLASSES 
    spec = spec[spec.normal_class != 'st'] # removing "stuk" class
    spec = spec[spec.normal_class != 'vuil'] # removing "vuil" class
    spec = spec[spec.normal_class.apply(lambda x: '+' not in str(x))]
    # SELECTING WANTED CLASSES
    # spec = spec[spec.normal_class.isin(['m','v','bl','c','wmv','v(cy)'])]
    # spec = spec[spec.normal_class.isin(['m','v','bl','c','wmv','v(cy)',
    # 									'bv','gaasvlieg','grv','k','kever','nl','psylloidea','sp','sst','sw','t','vlieg','weg','wnv','wswl'])]
    # spec = spec[spec.normal_class.isin(['m','v','bl','c','wmv','v(cy)',
    # 									'bv','sw','t'])]
    spec = spec[spec.normal_class.isin(['m','v','c','wmv','v(cy)','t'])]

    spec_nr_classes = spec['yolo_class'].unique().shape[0]
    condition1 = (spec_nr_classes >= 1)
    condition2 = True # (spec['yolo_class'].unique()[0] not in [nan_code, np.nan])
    condition3 = (spec['yolo_class'].isnull().sum() != spec['yolo_class'].shape[0])

    df_stats.loc[pname] = pd.Series({'nr_nans': spec[spec['yolo_class'] == nan_code].shape[0], 
                                        'unique_insects': spec['yolo_class'][spec['yolo_class'] != nan_code].unique().shape[0],
                                        'annotated': False})

    # finding the annotated plates - i.e the ones that don't have all nans in 'class'
    if condition1 and condition2 and condition3:
        print(f'\nFound annotated data: {condition1 and condition2} ----> COPYING IT')
        annotated_plates.append(platename)
        print(platename)
        spec['pname'] = pname

        spec['yolo_class'].fillna(0, inplace=True)
        spec['yolo_class'] = spec['yolo_class'].astype(int)
        spec['yolo_x'] = np.abs(spec['Bounding Rect Right.1'] - np.abs(spec['Bounding Rect Left.1'] - spec['Bounding Rect Right.1']) /2) / W
        spec['yolo_y'] = np.abs(spec['Bounding Rect Bottom.1'] - np.abs(spec['Bounding Rect Top.1'] - spec['Bounding Rect Bottom.1']) /2) / H
        spec['width'] = 150#np.abs(spec['Bounding Rect Left.1'] - spec['Bounding Rect Right.1']) + extra_pixels
        spec['height'] = 150#np.abs(spec['Bounding Rect Top.1'] - spec['Bounding Rect Bottom.1']) + extra_pixels

        # Making extracted boxes squares (to avoid distortions in future resizing)
        spec['yolo_width'] = pd.concat([spec['width'], spec['height']], axis=1).max(axis=1) / W 
        spec['yolo_height'] = pd.concat([spec['width'], spec['height']], axis=1).max(axis=1) / H


        ann_full_new = os.path.join( path_annotations , f"{pname}.txt" )
        img_full_new = os.path.join( path_images , pname ) + '.jpg'

        # SAVING IMAGES
        if not os.path.isfile( img_full_new ):
            print(f"Copying {pname}")
            cv2.imwrite(img_full_new, plate_img)
        # SAVING ANNOTATIONS
        if not len(spec) and not os.path.isfile( ann_full_new ):
            print('Empty file', ann_full_new)
            break
        else:#if not os.path.isfile( ann_full_new ):
            spec[['yolo_class','yolo_x','yolo_y','yolo_width','yolo_height']].to_csv(ann_full_new, sep=' ', index=False, header=False)

        df_stats.loc[pname] = pd.Series({'nr_nans': spec[spec['yolo_class'] == nan_code].shape[0], 
                                            'unique_insects': spec['yolo_class'][spec['yolo_class'] != nan_code].unique().shape[0],
                                            'annotated': True})

        all_specs.append(spec)

    else:
        incomplete_plates.append(platename)

df_specs = pd.concat(all_specs, axis=0)

# SAVING DATAFRAMES WITH STATISTICS REGARDING THE PLATES AND BOUNDING BOXES
df_stats.to_csv(f'{created_data_path}/df_stats.csv')
df_specs.to_csv(f'{created_data_path}/df_specs.csv')

0it [00:00, ?it/s]


Found annotated data: True ----> COPYING IT
/home/kalfasyan/data/images/sticky_plates/2020/20200706/plates/results/HerentCONTROLE_w04_F10_51 mm_ISO160_1-15 s_IncandescentWB.png
Copying HerentCONTROLE_w04_F10_51 mm_ISO160_1-15 s_IncandescentWB


1it [00:01,  1.22s/it]


Found annotated data: True ----> COPYING IT
/home/kalfasyan/data/images/sticky_plates/2020/20200706/plates/results/HerentCONTROLE_w05_F10_51 mm_ISO160_1-15 s_IncandescentWB.png
Copying HerentCONTROLE_w05_F10_51 mm_ISO160_1-15 s_IncandescentWB


2it [00:01,  1.08s/it]

...

22it [00:10,  1.52it/s]


Found annotated data: True ----> COPYING IT
/home/kalfasyan/data/images/sticky_plates/2020/20200706/plates/results/Herent_w26_centroid_F10_51 mm_ISO160_1-15 s_IncandescentWB.png
Copying Herent_w26_centroid_F10_51 mm_ISO160_1-15 s_IncandescentWB


66it [00:18,  6.25it/s]


Found annotated data: True ----> COPYING IT
/home/kalfasyan/data/images/sticky_plates/2020/20200723/plates/results/Racour_20200713_B_F10_51 mm_ISO160_1-15 s_IncandescentWB.png
Copying Racour_20200713_B_F10_51 mm_ISO160_1-15 s_IncandescentWB


90it [00:22,  4.00it/s]


In [10]:
if yolo_to_voc:
    # CONVERTING LABELS FROM YOLO ANNOTATIONS (txt) TO VOC (xml)
    os.system("python yolo_to_voc.py")
    # EXTRACTING BOUNDING BOXES AS IMAGES
    if extract_boxes:
        import sys
        sys.path.insert(0, '..')
        from impy.ObjectDetectionDataset import ObjectDetectionDataset
        sticky = ObjectDetectionDataset(imagesDirectory=path_images, 
                                        annotationsDirectory=path_voc_annotations,
                                        databaseName='sticky_plates')
        sticky.saveBoundingBoxes(outputDirectory=path_impy_crops_export)

  0%|          | 0/13 [00:00<?, ?it/s]../impy/ImageAnnotation.py:76: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if (root.find("size")):
../impy/ImageAnnotation.py:49: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if (root.find("object")):


HerentCONTROLE_w04_F10_51 mm_ISO160_1-15 s_IncandescentWB
img:/home/kalfasyan/projects/yannis_impy/insectrec/created_data/images/HerentCONTROLE_w04_F10_51 mm_ISO160_1-15 s_IncandescentWB.jpg


  8%|▊         | 1/13 [00:00<00:03,  3.46it/s]

HerentCONTROLE_w05_F10_51 mm_ISO160_1-15 s_IncandescentWB
img:/home/kalfasyan/projects/yannis_impy/insectrec/created_data/images/HerentCONTROLE_w05_F10_51 mm_ISO160_1-15 s_IncandescentWB.jpg


 15%|█▌        | 2/13 [00:00<00:03,  3.50it/s]

...

 85%|████████▍ | 11/13 [00:03<00:00,  3.15it/s]

Herent_w26_centroid_F10_51 mm_ISO160_1-15 s_IncandescentWB
img:/home/kalfasyan/projects/yannis_impy/insectrec/created_data/images/Herent_w26_centroid_F10_51 mm_ISO160_1-15 s_IncandescentWB.jpg


 92%|█████████▏| 12/13 [00:03<00:00,  3.23it/s]

Racour_20200713_B_F10_51 mm_ISO160_1-15 s_IncandescentWB
img:/home/kalfasyan/projects/yannis_impy/insectrec/created_data/images/Racour_20200713_B_F10_51 mm_ISO160_1-15 s_IncandescentWB.jpg


100%|██████████| 13/13 [00:04<00:00,  3.20it/s]
